# Explanatory Data Analysis
Перед нами стоит задача провести EDA. В первую очередь мы попробуем прочитать предложенный файл и посмотреть на результат. Но перед эти импортируем необходимые для этого библиотеки.


In [3]:
!pip install plotly
!pip install cufflinks
!pip install textblob
!pip install spacy

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
%matplotlib inline
import plotly as py
import cufflinks as cf
from plotly.offline import iplot
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [8]:
info = pd.read_csv('D:/Проекты/Analysis/texts.csv')
info.head()

,id,txt,gender,birthdate,create_date,source,additional_info
0,3181,Здравствуйте Евгений Альбертович! В октябре 20...,NaN,NaN,2021-04-10,vladmedicina.ru/faq/answers/73190.htm,NaN
1,3182,Здравствуйте. Как на этом сайте оставить отзыв?,NaN,NaN,2021-04-06,vladmedicina.ru/faq/answers/73172.htm,NaN
2,3183,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,NaN,2021-04-02,vladmedicina.ru/faq/answers/73154.htm,NaN
3,3184,"Здравствуйте, Алексей Леонтьевич! Скажите, пос...",NaN,NaN,2020-12-08,vladmedicina.ru/faq/answers/72437.htm,NaN
4,3185,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,NaN,2021-03-31,vladmedicina.ru/faq/answers/73141.htm,NaN


Далее, для упрощения анализа оставим только необходимые для этого столбцы. Для этого воспользуемся методом drop().

In [9]:
info.drop(labels=['id', 'birthdate', 'additional_info', 'source'], axis=1, inplace=True)
info.head()

,txt,gender,create_date
0,Здравствуйте Евгений Альбертович! В октябре 20...,NaN,2021-04-10
1,Здравствуйте. Как на этом сайте оставить отзыв?,NaN,2021-04-06
2,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,2021-04-02
3,"Здравствуйте, Алексей Леонтьевич! Скажите, пос...",NaN,2020-12-08
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31


Давайте посмотрим, есть ли в нашем наборе данных какие-либо нулевые значения:

In [10]:
info.isnull().sum()

txt               0
gender         5075
create_date       0
dtype: int64

Видно, что в столбце gender присутствуют пустые значения. Для того, чтобы было проще проводить анализ предложенных данных, воспользуемся методом dropna() для удаления нулевых значений.


In [11]:
info.dropna(subset=['gender'], inplace=True)
info.head()

,txt,gender,create_date
2,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,2021-04-02
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14


In [12]:
info.isnull().sum()

txt            0
gender         0
create_date    0
dtype: int64

Мы можем наглядно рассмотреть всевозможные сокращения, которые используются в русском языке и в последствии создадим словарь с такими сокращениями, дабы текст был более приятным на вид и понятным.


In [14]:
contraction = {"д/сад": "детский сад", "г.": "город", "мм": "миллиметр", "т.д": "так далее", "см": "сантиметр",
               "таб/день": "таблетки в день", "кг": "киллограмм", "уд/мин": "ударов в минуту"}

Напишем функцию, позволяющую применить наш словарь.

In [15]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contraction:
            value = contraction[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [16]:
info['txt'] = info['txt'].apply(lambda x: cont_to_exp(x))

# Очистка текстовых данных в Python

Как правило, текстовые данные содержат много шума либо в виде символов, либо в виде знаков препинания и стоп-слов. Следовательно, возникает необходимость очистить текст не только для того, чтобы сделать его более понятным, но и для лучшего понимания.

Мы можем видеть, что у нас в тексте есть нежелательные сокращения по типу 3.5 года, то есть цифры, которые можно записать в виде текста, а также знаки препинания.

# Нижний регистр

В НЛП модели трактуют такие слова, как график(план работы) и график(художник), по-разному, даже если они одинаковы. Поэтому, чтобы решить эту проблему, мы пишем слова в нижнем регистре. Здесь я использую функцию lower (), доступную в Python, для преобразования текста в нижний регистр:


In [17]:
info['cleaned']=info['txt'].apply(lambda x: x.lower())
info['cleaned']

2       добрый день, евгений альбертович!\nребенку 3,5...
4       здравствуйте! хочу оставить отзыв. восемь лет ...
5       здравствуйте! в 2018 году ребенку (сейчас ему ...
8       здравствуйте! скажите, пожалуйста, может ли ми...
12      здравствуйте, евгений альбертович! куда и с ка...
                              ...                        
6856    здравствуйте! беспокою вас с сахалина, пореком...
6861    здравствуйте! у меня муж ночью сильно храпит. ...
6867    здравствуйте. подскажите когда и где в ближайш...
6871    здравствуйте евгений альбертович! у меня к вам...
6875    в какую больницу лучше обратиться для исправле...
Name: cleaned, Length: 1801, dtype: object

# Удаление цифр и слов, содержащих цифры
Далее нам нужно удалить числа и слова, содержащие цифры. Я делаю это потому, что цифры и слова, содержащие цифры, не придают большого значения основным словам. Для этого я использую регулярные выражения с лямбда-функциями.


In [18]:
info['cleaned']=info['cleaned'].apply(lambda x: re.sub('\w*\d\w*','', x))

# Удаление пунктуации
Пунктуация - это знаки языка, такие как запятые, дефисы, точки и т.д. Они важны для грамматики языка, но не для анализа текста. Поэтому их нужно удалить:

In [19]:
info['cleaned']=info['cleaned'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

У нас остались лишние пробелы, удалим их

In [20]:
info['cleaned']=info['cleaned'].apply(lambda x: re.sub(' +',' ',x))
info['cleaned']=info['cleaned'].apply(lambda x: re.sub("^\s+|\n|\r|\s+$", ' ', x))

# Подготовка текстовых данных для исследовательского анализа данных (EDA)
Мы уже очистили наши данные и подготовили корпус. В этом разделе мы создадим матрицу терминов документа, которую позже будем использовать в нашем анализе.

В этом разделе сделаем следующее:

Удаление стоп-слов

Лемматизация

# Стоп-слова 
Стоп-слова - это наиболее распространенные слова в таких языках, как «я», «это», «есть», «в», которые не придают большого значения значению документа. Эти значения будут удалены, чтобы уменьшить размер набора данных и сосредоточить внимание на значимых словах.

# Лемматизация

Лемматизация - это систематический процесс сведения токена к его лемме. Он использует словарь, структуру слова, часть речевых тегов и грамматические отношения для преобразования слова в его базовую форму.

Для этого я буду использовать SpaCy для удаления игнорируемых слов и лемматизации.

In [21]:
import spacy
from spacy.lang.ru import Russian
nlp = Russian()
nlp = spacy.load('ru_core_news_sm',disable=['parser', 'ner'])
info['lemmatized']=info['cleaned'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
info['lemmatized'].head()
info_grouped=info[['txt','lemmatized']].groupby(by='txt').agg(lambda x:' '.join(x))
info_grouped.head()

,lemmatized
txt,
"1. Андрей м, 02.02.2013, 22:00\r\nЗдравствуйте ! У меня был инфаркт 1,5 месяца назад. Врачи предложили пройти коронографию, но у меня аллергия на обезболивающие убистезин, ультракаин с-п, септанест, ултракаин д-с форте. Что мне делать?\r\nНасколько болезненна коронография?",андрей м здравствовать инфаркт месяц наза...
"1. Андрей м, 02.02.2013, 22:00\r\nЗдравствуйте Валерий Гамитович! У меня был инфаркт 1,5 месяца назад. Врачи предложили пройти коронографию, но у меня аллергия на обезболивающие убистезин, ультракаин с-п, септанест, ултракаин д-с форте. Что мне делать?\r\nНасколько болезненна коронография?",андрей м здравствовать валерий гамитович ...
"1.Порвался презерватив...через месяц узнал, что у партнерши нашли Кондиломы. Это опасно для меня? Какие анализы нужно сдать для того, чтобы провериться максимально, ""чист"" ли я?2.имеется варикоцеле рецидивповторять операцию по Иванисевичу нет сантиметрысла же? Нужна эндоскопическая операция?",порваться презервативчерез месяц узнать партнё...
"130 С детства частое мочеиспускание , но тем не менее жизни это не мешало. Спокойно сидела и на уроках и на парах, и в кино. Если нужно было - выходила во время сеанса. Могла терпеть. Лишь только после много выпитой жидкости позывы были сильные . Ночами ходила раз или два. Порции небольшие. В марте прошлого месяца впервые был приступ - сильного непроходящего позыва , все анализы были в норме , узи тоже . Как то потом отпустило на время. и Вот в июле этого года снова все повторилось. Начался один сплошной непроходящий позыв . И снова ОАМ , БАК ПОСЕВ, ПО НИЧЕПОРЕНКО - ВСЕ В ИДЕАЛЕ. ПЦР МОЧИ тоже. Узи по женски и мочеполовой системы тоже без потологий. При цистоскопии объем мочевого пузыря 200 мл, стенки ровные и розовые , что позволило врачу предположить, что это не ИЦ. Нашли лейкоплакию на шейке МП 2,0 на 1,5 сантиметр. 21 августа прооперировалась. Немного боль при мочеиспускании , но позыв так и сохранился. Особенно чувствительно при ходьбе. Очень бъет по нервам.Черезе время позыв должен пройти после операции. Я понимаю, . что реже , чем раньше я ходить не буду , меня интересует именно позыв неприкращаюшийся. Пробовала пить везикар неделю, но мне не совсем хорошо от него , далее уротол почти месяц, подключила бетмигу ( все это до операции ) , действий не почувствовала . Они лишь как то тормозили сам процесс мочеиспускания. На женские гормоны сдала , немного повышен тестостерон. Все остальные показатели для 32 лет для 4 го дня цикла в норме. Возможен ли был этот позыв от лейкоплакии ? и как долго ждать его прекращения после операции? или же все же это ГАПМ ( но даже с учетом, что я ходила раньше часто в туалет ) почему он из частых позывов превратился в один сплошной позыв ? и судя по форумам , если это ГАМП -то он практически не лечится",детство частый мочеиспускание менее жизнь ме...
"4 года назад мне удалили матку из-за миомы. вчера часть шва покраснела, появился дискомфорт и боль при касании именно на этом участке. Это опухоль? к какому врачу обратится и какое пройти обследование? мне страшно",год назад удалить матка изза миома вчера час...


Посчитаем базовые статистики по текстам

In [22]:
from textblob import TextBlob
info['review_len'] = info['lemmatized'].apply(lambda x: len(x))  # Считаем количество символов
info['w_count'] = info['lemmatized'].apply(lambda x: len(x.split()))  # Общее количество слов в данном наборе

In [23]:
# Средняя длина слова
def middle_words(x):
    words = x.split()
    word_len = 0
    for word in words:
        word_len = word_len + len(word)
    return word_len/len(words)

In [24]:
info['middle_words'] = info['lemmatized'].apply(lambda x: middle_words(x))  # Поиск средней длины слова
info.head()

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
2,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,2021-04-02,добрый день евгений альбертович ребенку года с...,добрый день евгений альбертович ребёнок год ст...,332,46,6.239130
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31,здравствуйте хочу оставить отзыв восемь лет мо...,здравствовать хотеть оставить отзыв восемь год...,530,59,8.000000
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29,здравствуйте в году ребенку сейчас ему лет уда...,здравствовать год ребёнок сейчас год удалить а...,274,34,6.852941
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24,здравствуйте скажите пожалуйста может ли минол...,здравствовать сказать пожалуйста ли минолексин...,122,15,7.200000
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14,здравствуйте евгений альбертович куда и с каки...,здравствовать евгений альбертович куда какой о...,157,20,6.900000


# Визуализация
Визуализируем полученные результаты

In [25]:
info['review_len'].iplot(kind='hist', xTitle = "Количество символов", yTitle = "Количество отзывов, имеющих столько символов", title = "Количество символов")

In [26]:
info['w_count'].iplot(kind='hist', xTitle = "Количество слов", yTitle = "Количество отзывов, имеющих столько слов", title = "Количество слов")

In [27]:
info['middle_words'].iplot(kind='hist', xTitle = "Средняя длина слов", yTitle = "Количество отзывов, имеющих такую среднюю длину слова", title = "Средняя длина слов")

Теперь попробуем отыскать самые часто встречающиеся слова в нашем файле в принципе, после чего соберем самые популярные слова от разных полов.

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
def top_words(x,n):
    vec = CountVectorizer().fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    w = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    w = sorted(w, key = lambda x: x[1], reverse = True)
    return w[:n]
words = top_words(info['lemmatized'], 15)
words

[('год', 1104),
 ('здравствовать', 912),
 ('ли', 896),
 ('день', 720),
 ('операция', 604),
 ('спасибо', 508),
 ('сказать', 507),
 ('врач', 474),
 ('делать', 472),
 ('можно', 463),
 ('какой', 445),
 ('пожалуйста', 443),
 ('добрый', 435),
 ('после', 394),
 ('лечение', 390)]

In [29]:
kek = info[info.gender=='male']
kek

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
16,"Евгений Альбертович, здравствуйте! У меня одно...",male,2021-01-31,евгений альбертович здравствуйте у меня одност...,евгений альбертович здравствовать односторонни...,389,45,7.666667
31,Евгений Альбертович! после операции по удалени...,male,2020-11-16,евгений альбертович после операции по удалению...,евгений альбертович после операция удаление ха...,329,37,7.918919
41,"Здравствуйте, хочу записаться к вам на прием ,...",male,2020-10-05,здравствуйте хочу записаться к вам на прием ко...,здравствовать хотеть записаться приём консульт...,131,14,8.428571
58,"Здравствуйте, Виктор Григорьевич!\r\nСпасибо В...",male,2020-03-29,здравствуйте виктор григорьевич спасибо вам б...,здравствовать виктор григорьевич спасибо бол...,814,97,7.298969
67,Виктор Г ригорьевич добрый день. болею с конца...,male,2015-10-14,виктор г ригорьевич добрый день болею с конца ...,виктор г ригорьевич добрый день болеть конец а...,212,26,7.192308
...,...,...,...,...,...,...,...,...
6738,"Добрый день, Евгений Альбертович.\r\nЯ был у В...",male,2012-12-16,добрый день евгений альбертович я был у вас н...,добрый день евгений альбертович консультация...,1239,153,7.065359
6761,у меня была травма носа с 3 лет-искривление пе...,male,2012-11-02,у меня была травма носа с летискривление перег...,травма нос летискривление перегородкидва год н...,233,28,7.357143
6849,"Здравствуйте,в последнее время мне стало трудн...",male,2012-04-11,здравствуйтев последнее время мне стало трудно...,здравствуйтев последний время стать трудный ды...,135,16,7.500000
6856,"здравствуйте! беспокою вас с сахалина, пореком...",male,2012-04-02,здравствуйте беспокою вас с сахалина порекомен...,здравствовать беспокою сахалин порекомендовать...,266,30,7.900000


In [30]:
man = top_words(kek['lemmatized'], 15)
man

[('здравствовать', 255),
 ('год', 247),
 ('ли', 241),
 ('операция', 162),
 ('день', 150),
 ('сказать', 125),
 ('можно', 121),
 ('добрый', 120),
 ('спасибо', 118),
 ('какой', 113),
 ('делать', 109),
 ('лечение', 104),
 ('сделать', 102),
 ('после', 100),
 ('пожалуйста', 100)]

In [31]:
puk = info[info.gender=='female']
puk

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
2,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,2021-04-02,добрый день евгений альбертович ребенку года с...,добрый день евгений альбертович ребёнок год ст...,332,46,6.239130
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31,здравствуйте хочу оставить отзыв восемь лет мо...,здравствовать хотеть оставить отзыв восемь год...,530,59,8.000000
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29,здравствуйте в году ребенку сейчас ему лет уда...,здравствовать год ребёнок сейчас год удалить а...,274,34,6.852941
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24,здравствуйте скажите пожалуйста может ли минол...,здравствовать сказать пожалуйста ли минолексин...,122,15,7.200000
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14,здравствуйте евгений альбертович куда и с каки...,здравствовать евгений альбертович куда какой о...,157,20,6.900000
...,...,...,...,...,...,...,...,...
6848,"Здравствуйте, обратилась к терапевту с болью в...",female,2012-04-13,здравствуйте обратилась к терапевту с болью в ...,здравствовать обратиться терапевт боль горлеор...,428,50,7.580000
6855,Здравствуйте Евгений Альбертович. Я из Находки...,female,2012-04-02,здравствуйте евгений альбертович я из находки ...,здравствовать евгений альбертович находка зват...,456,54,7.462963
6861,Здравствуйте! У меня муж ночью сильно храпит. ...,female,2012-03-25,здравствуйте у меня муж ночью сильно храпит чт...,здравствовать муж ночь сильно храпеть можно сд...,82,11,6.545455
6867,Здравствуйте. Подскажите когда и где в ближайш...,female,2012-02-22,здравствуйте подскажите когда и где в ближайше...,здравствовать подсказать где близкий время мож...,70,9,6.888889


In [32]:
girl = top_words(puk['lemmatized'], 15)
girl

[('год', 857),
 ('здравствовать', 657),
 ('ли', 655),
 ('день', 570),
 ('операция', 442),
 ('спасибо', 390),
 ('сказать', 382),
 ('врач', 378),
 ('делать', 363),
 ('пожалуйста', 343),
 ('можно', 342),
 ('какой', 332),
 ('ребёнок', 320),
 ('добрый', 315),
 ('после', 294)]

# Юниграммы

In [33]:
lol = pd.DataFrame(words, columns = ['Unigram', 'Frequency'])
lol = lol.set_index('Unigram')
lol.iplot(kind='bar', xTitle='Юниграмма', yTitle='Количество', title='Юниграмма общая')

In [34]:
mem = pd.DataFrame(man, columns = ['Unigram', 'Frequency'])
mem = mem.set_index('Unigram')
mem.iplot(kind='bar', xTitle='Юниграмма', yTitle='Количество', title='Юниграмма мужская')

In [35]:
kuk = pd.DataFrame(girl, columns = ['Unigram', 'Frequency'])
kuk = kuk.set_index('Unigram')
kuk.iplot(kind='bar', xTitle='Юниграмма', yTitle='Количество', title='Юниграмма женская')

# Биграммы

In [36]:
def top_words(x,n):
    vec = CountVectorizer(ngram_range=(2,2)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    w = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    w = sorted(w, key = lambda x: x[1], reverse = True)
    return w[:n]

In [37]:
words = top_words(info['lemmatized'], 20)
words

[('добрый день', 330),
 ('подсказать пожалуйста', 207),
 ('евгений альбертович', 192),
 ('заранее спасибо', 143),
 ('можно ли', 132),
 ('год назад', 120),
 ('поставить диагноз', 113),
 ('здравствовать евгений', 105),
 ('сказать пожалуйста', 101),
 ('возможный ли', 86),
 ('попасть приём', 79),
 ('спасибо ответ', 69),
 ('врач сказать', 64),
 ('нужный ли', 57),
 ('ребёнок год', 56),
 ('сделать операция', 53),
 ('ваш ответ', 52),
 ('делать операция', 51),
 ('после операция', 48),
 ('сын год', 47)]

In [38]:
ustal = pd.DataFrame(words, columns = ['Bigram', 'Frequency'])
ustal = ustal.set_index('Bigram')
ustal.iplot(kind='bar', xTitle='Биграмма', yTitle='Количество', title='Биграмма общая')

In [39]:
kek = info[info.gender=='male']
kek

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
16,"Евгений Альбертович, здравствуйте! У меня одно...",male,2021-01-31,евгений альбертович здравствуйте у меня одност...,евгений альбертович здравствовать односторонни...,389,45,7.666667
31,Евгений Альбертович! после операции по удалени...,male,2020-11-16,евгений альбертович после операции по удалению...,евгений альбертович после операция удаление ха...,329,37,7.918919
41,"Здравствуйте, хочу записаться к вам на прием ,...",male,2020-10-05,здравствуйте хочу записаться к вам на прием ко...,здравствовать хотеть записаться приём консульт...,131,14,8.428571
58,"Здравствуйте, Виктор Григорьевич!\r\nСпасибо В...",male,2020-03-29,здравствуйте виктор григорьевич спасибо вам б...,здравствовать виктор григорьевич спасибо бол...,814,97,7.298969
67,Виктор Г ригорьевич добрый день. болею с конца...,male,2015-10-14,виктор г ригорьевич добрый день болею с конца ...,виктор г ригорьевич добрый день болеть конец а...,212,26,7.192308
...,...,...,...,...,...,...,...,...
6738,"Добрый день, Евгений Альбертович.\r\nЯ был у В...",male,2012-12-16,добрый день евгений альбертович я был у вас н...,добрый день евгений альбертович консультация...,1239,153,7.065359
6761,у меня была травма носа с 3 лет-искривление пе...,male,2012-11-02,у меня была травма носа с летискривление перег...,травма нос летискривление перегородкидва год н...,233,28,7.357143
6849,"Здравствуйте,в последнее время мне стало трудн...",male,2012-04-11,здравствуйтев последнее время мне стало трудно...,здравствуйтев последний время стать трудный ды...,135,16,7.500000
6856,"здравствуйте! беспокою вас с сахалина, пореком...",male,2012-04-02,здравствуйте беспокою вас с сахалина порекомен...,здравствовать беспокою сахалин порекомендовать...,266,30,7.900000


In [40]:
man = top_words(kek['lemmatized'], 20)
man

[('добрый день', 91),
 ('подсказать пожалуйста', 42),
 ('евгений альбертович', 41),
 ('заранее спасибо', 34),
 ('сказать пожалуйста', 33),
 ('можно ли', 29),
 ('поставить диагноз', 29),
 ('год назад', 28),
 ('возможный ли', 27),
 ('сделать операция', 22),
 ('здравствовать евгений', 22),
 ('сколько стоить', 20),
 ('попасть приём', 19),
 ('операция удаление', 16),
 ('записаться приём', 15),
 ('после операция', 14),
 ('сколько стоять', 14),
 ('нужный ли', 14),
 ('юрий александрович', 14),
 ('месяц назад', 13)]

In [41]:
ustal = pd.DataFrame(man, columns = ['Bigram', 'Frequency'])
ustal = ustal.set_index('Bigram')
ustal.iplot(kind='bar', xTitle='Биграмма', yTitle='Количество', title='Биграмма мужская')

In [42]:
puk = info[info.gender=='female']
puk

,txt,gender,create_date,cleaned,lemmatized,review_len,w_count,middle_words
2,"Добрый день, Евгений Альбертович!\nРебенку 3,5...",female,2021-04-02,добрый день евгений альбертович ребенку года с...,добрый день евгений альбертович ребёнок год ст...,332,46,6.239130
4,Здравствуйте! Хочу оставить отзыв. Восемь лет ...,female,2021-03-31,здравствуйте хочу оставить отзыв восемь лет мо...,здравствовать хотеть оставить отзыв восемь год...,530,59,8.000000
5,Здравствуйте! В 2018 году ребенку (сейчас ему ...,female,2021-03-29,здравствуйте в году ребенку сейчас ему лет уда...,здравствовать год ребёнок сейчас год удалить а...,274,34,6.852941
8,"Здравствуйте! Скажите, пожалуйста, может ли ми...",female,2021-02-24,здравствуйте скажите пожалуйста может ли минол...,здравствовать сказать пожалуйста ли минолексин...,122,15,7.200000
12,"Здравствуйте, Евгений Альбертович! Куда и с ка...",female,2021-02-14,здравствуйте евгений альбертович куда и с каки...,здравствовать евгений альбертович куда какой о...,157,20,6.900000
...,...,...,...,...,...,...,...,...
6848,"Здравствуйте, обратилась к терапевту с болью в...",female,2012-04-13,здравствуйте обратилась к терапевту с болью в ...,здравствовать обратиться терапевт боль горлеор...,428,50,7.580000
6855,Здравствуйте Евгений Альбертович. Я из Находки...,female,2012-04-02,здравствуйте евгений альбертович я из находки ...,здравствовать евгений альбертович находка зват...,456,54,7.462963
6861,Здравствуйте! У меня муж ночью сильно храпит. ...,female,2012-03-25,здравствуйте у меня муж ночью сильно храпит чт...,здравствовать муж ночь сильно храпеть можно сд...,82,11,6.545455
6867,Здравствуйте. Подскажите когда и где в ближайш...,female,2012-02-22,здравствуйте подскажите когда и где в ближайше...,здравствовать подсказать где близкий время мож...,70,9,6.888889


In [43]:
girl = top_words(puk['lemmatized'], 20)
girl

[('добрый день', 239),
 ('подсказать пожалуйста', 165),
 ('евгений альбертович', 151),
 ('заранее спасибо', 109),
 ('можно ли', 103),
 ('год назад', 92),
 ('поставить диагноз', 84),
 ('здравствовать евгений', 83),
 ('сказать пожалуйста', 68),
 ('попасть приём', 60),
 ('возможный ли', 59),
 ('спасибо ответ', 57),
 ('врач сказать', 51),
 ('ребёнок год', 48),
 ('ваш ответ', 44),
 ('сын год', 43),
 ('нужный ли', 43),
 ('делать операция', 40),
 ('можно сделать', 35),
 ('заранее благодарить', 35)]

In [44]:
ustal = pd.DataFrame(girl, columns = ['Bigram', 'Frequency'])
ustal = ustal.set_index('Bigram')
ustal.iplot(kind='bar', xTitle='Биграмма', yTitle='Количество', title='Биграмма женская')

# Триграммы

In [45]:
def top_words(x,n):
    vec = CountVectorizer(ngram_range=(3,3)).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    w = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    w = sorted(w, key = lambda x: x[1], reverse = True)
    return w[:n]

In [46]:
words = top_words(info['lemmatized'], 10)
words

[('здравствовать евгений альбертович', 82),
 ('добрый день евгений', 39),
 ('день евгений альбертович', 37),
 ('заранее спасибо ответ', 37),
 ('добрый день подсказать', 34),
 ('здравствовать валерий гамитович', 33),
 ('день подсказать пожалуйста', 31),
 ('здравствовать подсказать пожалуйста', 30),
 ('спасибо ваш ответ', 28),
 ('год рост вес', 27)]

In [47]:
ustal = pd.DataFrame(words, columns = ['Trigram', 'Frequency'])
ustal = ustal.set_index('Trigram')
ustal.iplot(kind='bar', xTitle='Триграмма', yTitle='Количество', title='Триграмма общая')

In [48]:
kek = info[info.gender=='male']
man = top_words(kek['lemmatized'], 10)
man

[('здравствовать евгений альбертович', 17),
 ('добрый время суток', 10),
 ('spp обнаружить обнаружить', 10),
 ('здравствовать валерий гамитович', 9),
 ('заранее спасибо ответ', 8),
 ('обрезание крайний плоть', 8),
 ('добрый день евгений', 8),
 ('добрый день год', 7),
 ('можно попасть приём', 7),
 ('здравствовать юрий александрович', 7)]

In [49]:
ustal = pd.DataFrame(man, columns = ['Trigram', 'Frequency'])
ustal = ustal.set_index('Trigram')
ustal.iplot(kind='bar', xTitle='Триграмма', yTitle='Количество', title='Триграмма мужская')

In [50]:
puk = info[info.gender=='female']
girl = top_words(puk['lemmatized'], 10)
girl

[('здравствовать евгений альбертович', 65),
 ('добрый день евгений', 31),
 ('день евгений альбертович', 30),
 ('заранее спасибо ответ', 29),
 ('добрый день подсказать', 28),
 ('спасибо ваш ответ', 28),
 ('год рост вес', 27),
 ('день подсказать пожалуйста', 27),
 ('кровь липидный спектр', 27),
 ('здравствовать подсказать пожалуйста', 26)]

In [51]:
ustal = pd.DataFrame(man, columns = ['Trigram', 'Frequency'])
ustal = ustal.set_index('Trigram')
ustal.iplot(kind='bar', xTitle='Триграмма', yTitle='Количество', title='Триграмма женская')